In [1]:
import numpy as np
import os 
import pandas as pd

In [2]:
def camb_clfile(fname, lmax=None):
    """CAMB spectra (lenspotentialCls, lensedCls or tensCls types) returned as a dict of numpy arrays.
    Args:
        fname (str): path to CAMB output file
        lmax (int, optional): outputs cls truncated at this multipole.
    """
    cols = np.loadtxt(fname).transpose()
    ell = np.int_(cols[0])
    if lmax is None: lmax = ell[-1]
    assert ell[-1] >= lmax, (ell[-1], lmax)
    cls = {k : np.zeros(lmax + 1, dtype=float) for k in ['tt', 'ee', 'bb', 'te']}
    w = ell * (ell + 1) / (2. * np.pi)  # weights in output file
    idc = np.where(ell <= lmax) if lmax is not None else np.arange(len(ell), dtype=int)
    for i, k in enumerate(['tt', 'ee', 'bb', 'te']):
        cls[k][ell[idc]] = cols[i + 1][idc] / w[idc]
    if len(cols) > 5:
        wpp = lambda ell : ell ** 2 * (ell + 1) ** 2 / (2. * np.pi)
        wptpe = lambda ell : np.sqrt(ell.astype(float) ** 3 * (ell + 1.) ** 3) / (2. * np.pi)
        for i, k in enumerate(['pp', 'pt', 'pe']):
            cls[k] = np.zeros(lmax + 1, dtype=float)
        cls['pp'][ell[idc]] = cols[5][idc] / wpp(ell[idc])
        cls['pt'][ell[idc]] = cols[6][idc] / wptpe(ell[idc])
        cls['pe'][ell[idc]] = cols[7][idc] / wptpe(ell[idc])
    return cls

In [3]:
path = "Sims/CAMB/"
len_file = "BBSims_lensed_dls.dat"
unlen_file =  "BBSims_scal_dls.dat"
pot_file = "BBSims_lenspotential.dat"

In [4]:
sca = os.path.join(path,unlen_file)
with open(sca) as f:
    firstline = next(f)

In [5]:
keys = [i.lower() for i in firstline.split(' ') if i.isalpha()][1:]   

In [6]:
keys

['tt', 'ee', 'te', 'pp', 'tp']

In [43]:
def camb_clfile2(fname, lmax=None):
    with open(fname) as f:
        firstline = next(f)
    keys = [i.lower() for i in firstline.split(' ') if i.isalpha()][1:]
    cols = np.loadtxt(fname).transpose()
    
    ell = np.int_(cols[0])
    if lmax is None: lmax = ell[-1]
    assert ell[-1] >= lmax, (ell[-1], lmax)
    
    cls = {k : np.zeros(lmax + 1, dtype=float) for k in keys}
    
    idc = np.where(ell <= lmax) if lmax is not None else np.arange(len(ell), dtype=int)
    
    w = lambda ell :ell * (ell + 1) / (2. * np.pi)
    wpp = lambda ell : ell ** 2 * (ell + 1) ** 2 / (2. * np.pi)
    wptpe = lambda ell :np.sqrt(ell.astype(float) ** 3 * (ell + 1.) ** 3) / (2. * np.pi) 
    for i, k in enumerate(keys):
        if k == 'pp':
            we = wpp(ell)
        elif 'p' in k and ('e' in k or 't' in k):
            we = wptpe(ell)
        else:
            we = w(ell)
        cls[k][ell[idc]] = cols[i + 1][idc] / we[idc]
    return cls

In [48]:
camb_clfile(os.path.join(path,len_file),lmax=10)

{'tt': array([   0.        ,    0.        , 1069.29341953,  506.20848946,
         287.52892691,  183.61059208,  127.21670011,   93.53643327,
          71.86244842,   57.20330586,   46.75638058]),
 'ee': array([0.        , 0.        , 0.03255989, 0.02091306, 0.01090585,
        0.00486777, 0.00194988, 0.00079019, 0.00039518, 0.00025339,
        0.00017723]),
 'bb': array([0.00000000e+00, 0.00000000e+00, 1.91781665e-06, 1.91635057e-06,
        1.91550758e-06, 1.91524473e-06, 1.91538416e-06, 1.91575442e-06,
        1.91628425e-06, 1.91691606e-06, 1.91765100e-06]),
 'te': array([0.        , 0.        , 2.75969018, 1.54635997, 0.87043793,
        0.49435683, 0.28456845, 0.16766456, 0.10298315, 0.06769511,
        0.04862866])}

In [47]:
camb_clfile2(os.path.join(path,len_file),lmax=10)

{'tt': array([   0.        ,    0.        , 1069.29341953,  506.20848946,
         287.52892691,  183.61059208,  127.21670011,   93.53643327,
          71.86244842,   57.20330586,   46.75638058]),
 'ee': array([0.        , 0.        , 0.03255989, 0.02091306, 0.01090585,
        0.00486777, 0.00194988, 0.00079019, 0.00039518, 0.00025339,
        0.00017723]),
 'bb': array([0.00000000e+00, 0.00000000e+00, 1.91781665e-06, 1.91635057e-06,
        1.91550758e-06, 1.91524473e-06, 1.91538416e-06, 1.91575442e-06,
        1.91628425e-06, 1.91691606e-06, 1.91765100e-06]),
 'te': array([0.        , 0.        , 2.75969018, 1.54635997, 0.87043793,
        0.49435683, 0.28456845, 0.16766456, 0.10298315, 0.06769511,
        0.04862866])}